In [38]:
import pandas as pd
import re

In [39]:
df_camera_front = pd.read_csv('camera_front.csv')
print(df_camera_front.head())

   product_id                                       camera_front
0       59254  12MP, ƒ/1.9, Tự động lấy nét theo pha Focus Pi...
1       76439  32MPƒ/2,0FOV 80,8˚Chế độ ban đêmHDRTính năng c...
2       59294                                       12 MP, f/2.2
3       59257  12MP, ƒ/1.9, Tự động lấy nét theo pha Focus Pi...
4       59258  12MP, ƒ/1.9, Tự động lấy nét theo pha Focus Pi...


In [40]:
# Chỉnh sửa hàm để trích xuất camera phụ nếu tìm thấy MP thứ hai
def extract_camera_attributes_with_secondary(camera_info):
    if not isinstance(camera_info, str):
        camera_info = ""
    
    megapixels = aperture = fov = hdr = focus = secondary_megapixels = None

    # Tìm kiếm megapixels (lần đầu tiên, camera chính)
    mp_matches = re.findall(r'(\d+)\s*MP', camera_info, re.IGNORECASE)
    if mp_matches:
        megapixels = mp_matches[0]  # MP đầu tiên tìm thấy là camera chính
        if len(mp_matches) > 1:
            secondary_megapixels = mp_matches[1]  # MP thứ hai tìm thấy là camera phụ

    # Tìm kiếm khẩu độ (ví dụ: 'ƒ/1.9', 'f/2.2')
    aperture_match = re.search(r'[ƒf]/(\d+\.?\d*)', camera_info)
    if aperture_match:
        aperture = aperture_match.group(1)

    # Tìm kiếm FOV (Trường nhìn) (ví dụ: 'FOV 80.8˚')
    fov_match = re.search(r'FOV\s*(\d+\.?\d*)', camera_info, re.IGNORECASE)
    if fov_match:
        fov = fov_match.group(1)
    
    return pd.Series([megapixels, secondary_megapixels, aperture, fov])

# Áp dụng hàm đã cập nhật để trích xuất cả thuộc tính camera chính và phụ
df_camera_front[['megapixels', 'secondary_megapixels', 'aperture', 'fov']] = df_camera_front['camera_front'].apply(extract_camera_attributes_with_secondary)

# Hiển thị DataFrame đã cập nhật
df_camera_front.head()


,product_id,camera_front,megapixels,secondary_megapixels,aperture,fov
0,59254,"12MP, ƒ/1.9, Tự động lấy nét theo pha Focus Pi...",12,None,1.9,None
1,76439,"32MPƒ/2,0FOV 80,8˚Chế độ ban đêmHDRTính năng c...",32,None,2,80
2,59294,"12 MP, f/2.2",12,None,2.2,None
3,59257,"12MP, ƒ/1.9, Tự động lấy nét theo pha Focus Pi...",12,None,1.9,None
4,59258,"12MP, ƒ/1.9, Tự động lấy nét theo pha Focus Pi...",12,None,1.9,None


In [41]:
# Xóa cột 'camera_front'
df_camera_front.drop(columns=['camera_front'], inplace=True)

# Điền những dữ liệu trống thành 'N/A'
df_camera_front.fillna('N/A', inplace=True)

# Hiển thị DataFrame đã cập nhật
df_camera_front.head()

,product_id,megapixels,secondary_megapixels,aperture,fov
0,59254,12,N/A,1.9,N/A
1,76439,32,N/A,2,80
2,59294,12,N/A,2.2,N/A
3,59257,12,N/A,1.9,N/A
4,59258,12,N/A,1.9,N/A


In [42]:
# Đổi tên các cột
df_camera_front.rename(columns={
    'megapixels': 'megapixels (MP)',
    'secondary_megapixels': 'secondary_megapixels (MP)',
    'aperture': 'wide_camera_mp (f/)',
    'fov': 'macro_camera_mp (radian)',
}, inplace=True)

In [43]:
# Save the modified DataFrame to a new CSV file
df_camera_front.to_csv('camera_front_processed.csv', index=False, encoding='utf-8-sig')